In [2]:
from datasets import load_dataset

In [3]:
# 457K rows
raw_datasets = load_dataset("code_search_net", "python")

Using the latest cached version of the module from /home/yangyansheng/.cache/huggingface/modules/datasets_modules/datasets/code_search_net/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1 (last modified on Mon Dec 18 10:40:29 2023) since it couldn't be found locally at code_search_net., or remotely on the Hugging Face Hub.
Found cached dataset code_search_net (/home/yangyansheng/.cache/huggingface/datasets/code_search_net/python/1.0.0/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
raw_datasets["train"].features

{'repository_name': Value(dtype='string', id=None),
 'func_path_in_repository': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'whole_func_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'func_code_string': Value(dtype='string', id=None),
 'func_code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'func_documentation_string': Value(dtype='string', id=None),
 'func_documentation_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'split_name': Value(dtype='string', id=None),
 'func_code_url': Value(dtype='string', id=None)}

In [5]:
print(raw_datasets["train"][1234]["whole_func_string"])

def _get_class_handlers(cls, signal_name, instance):
        """Returns the handlers registered at class level.
        """
        handlers = cls._signal_handlers_sorted[signal_name]
        return [getattr(instance, hname) for hname in handlers]


In [6]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

In [7]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_corpus = get_training_corpus()

tokenizer = old_tokenizer.train_new_from_iterator(train_corpus, 52000)

In [8]:
tokenizer.save_pretrained("./python_code_tokenizer")

('./python_code_tokenizer/tokenizer_config.json',
 './python_code_tokenizer/special_tokens_map.json',
 './python_code_tokenizer/vocab.json',
 './python_code_tokenizer/merges.txt',
 './python_code_tokenizer/added_tokens.json',
 './python_code_tokenizer/tokenizer.json')

In [9]:
import os

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

from transformers import AutoTokenizer, LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
import sentencepiece as spm
import argparse

In [10]:
chinese_sp_model = spm.SentencePieceProcessor()
chinese_sp_model.Load("./medical_sp.model")
chinese_spm = sp_pb2_model.ModelProto()
chinese_spm.ParseFromString(chinese_sp_model.serialized_model_proto())

389841

In [11]:
llama_tokenizer = AutoTokenizer.from_pretrained(
    "/home/yangyansheng/workspace/models/internlm-chat-7b",
    trust_remote_code=True,
)
llama_spm = sp_pb2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())

1658691

In [12]:
start = 256
pieces = llama_spm.pieces[start : start + 20]

for piece in pieces:
    print(piece.piece, piece.score)

<0xFD> 0.0
<0xFE> 0.0
<0xFF> 0.0
' -1635.0
b -1.0
e -2.0
▁ -1298.0
t -4.0
a -5.0
n -6.0
i -7.0
r -8.0
o -9.0
s -10.0
l -11.0
c -12.0
\ -2473.0
d -14.0
x -15.0
u -16.0
